<a href="https://colab.research.google.com/github/Praveengovianalytics/10MinAIAgentChallenge/blob/main/Smart_Travel_Agent_with_Agentic_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🧠 What is this?
Welcome to the #10MinAIAgentChallenge!
This notebook demonstrates how to build a smart, agentic travel planner using LangGraph, LangChain, and OpenAI GPT-4 – all within 10 minutes.

Whether you're at an airport lounge or brainstorming your next vacation, this AI agent will:

📍 Understand your destination & dates

📄 Create a 3-day trip outline

🔍 Auto-research top highlights using Tavily

📋 Generate a detailed daily itinerary

# Smart Travel Agent with Agentic AI

### Install necessary libraries

In [1]:
!pip install langchain-openai langgraph python-dotenv langchain-community -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.3/65.3 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.4/152.4 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 51.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.1/438.1 kB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.2/44.2 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.0/363.0 kB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.5/216.5 kB 15.6 MB/s eta 0:00:00


## Install necessary libraries

In [2]:
from google.colab import userdata
import os
from typing import TypedDict, List
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, START, END
from langchain_community.tools.tavily_search import TavilySearchResults

## Set your environment keys securely using userdata

In [7]:
# prompt: use os library to set env

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
os.environ["TAVILY_API_KEY"] = userdata.get('TAVILY_API_KEY')

# Setup GenAI model and Websearch tool

#### Setup account in TAVILY and create API key

In [10]:
llm = ChatOpenAI(model="gpt-4", temperature=0.7)
search_tool = TavilySearchResults(max_results=5, api_key=userdata.get('TAVILY_API_KEY'))

## Simple Agentic Workflow

In [16]:
class State(TypedDict):
    destination: str
    dates: str
    outline: str
    research: List[str]
    itinerary: str

# 1. Instantiate graph with state schema
graph = StateGraph(State)

# 2. Define node functions
def plan_node(state: State) -> dict:
    prompt = (
        f"You are expert travel planner. Create a 5‑day outline for "
        f"{state['destination']} ({state['dates']})."
    )
    outline = llm.predict(prompt)
    return {"outline": outline}

def research_node(state: State) -> dict:
    items = []
    for day in state["outline"].splitlines():
        q = f"{state['destination']} {day} highlights"
        snippet = search_tool.invoke(q)[0].get("snippet", "")
        items.append(snippet)
    return {"research": items}

def generate_node(state: State) -> dict:
    content = "\n".join(state["research"])
    prompt = (
        f"Outline:\n{state['outline']}\n\nResearch:\n{content}\n\n"
        "Generate a detailed day‑by‑day itinerary."
    )
    return {"itinerary": llm.predict(prompt)}

# 3. Add nodes to graph
graph.add_node("plan", plan_node)
graph.add_node("gather_research", research_node)   # ✅ renamed to avoid conflict
graph.add_node("generate", generate_node)

# 4. Connect nodes with edges
graph.add_edge(START, "plan")
graph.add_edge("plan", "gather_research")          # ✅ updated edge
graph.add_edge("gather_research", "generate")
graph.add_edge("generate", END)

# 5. Compile graph
planner = graph.compile()

# 6. Run
def main():
    state: State = {
        "destination": input("Destination: "),
        "dates": input("Dates (e.g. July 1‑5): "),
        "outline": "",
        "research": [],
        "itinerary": ""
    }
    final_state = planner.invoke(state)
    print("\n🗓️ Outline:\n", final_state["outline"])
    print("\n🔍 Research:\n", "\n".join(final_state["research"]))
    print("\n📋 Itinerary:\n", final_state["itinerary"])

if __name__ == "__main__":
    main()


🗓️ Outline:
 Day 1 - June 17:
- Morning: Arrival in Colombo. Check-in at the hotel and freshen up.
- Afternoon: Start your tour with a visit to the National Museum of Colombo to understand the rich history and culture of Sri Lanka.
- Evening: Visit the Gangaramaya Buddhist Temple, a mix of Indian, Thai, and Chinese architecture. 

Day 2 - June 18:
- Morning: Visit the Pettah Market, a bustling local market where you can buy everything from textiles to electronics and food items.
- Afternoon: Head to the Galle Face Green, a beautiful promenade along the Indian Ocean. Enjoy local food at the street stalls.
- Evening: Visit the Old Dutch Hospital, now a shopping and dining complex. Enjoy dinner at one of the restaurants there.

Day 3 - June 19:
- Morning: Visit the Colombo Dutch Museum and the Wolvendaal Church for some historical exploration.
- Afternoon: Visit the Viharamahadevi Park, the largest park in Colombo. 
- Evening: Free time for shopping or exploring the city at your own pace